<a href="https://colab.research.google.com/github/Vaibhavi-Sita/StudyAid/blob/dev/StudyAid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install cdqa

In [2]:
import os
import pandas as pd
from ast import literal_eval

from cdqa.utils.converters import pdf_converter
from cdqa.utils.filters import filter_paragraphs
from cdqa.pipeline import QAPipeline
from cdqa.utils.download import download_model

/usr/local/lib/python3.7/dist-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
# Download model
download_model(model='bert-squad_1.1', dir='./models')


bert_qa.joblib already downloaded


In [4]:
df = pdf_converter(directory_path='./data/pdf/')
df.head()

,title,paragraphs
0,Chapter 9 - Mobile Internet Devices and the Cloud,"[235 , Chapter 9 , Mobile Internet Devices an..."


In [ ]:
cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib', max_df=1.0)

# Fit Retriever to documents
cdqa_pipeline.fit_retriever(df=df)

In [6]:
query = 'How many connections to mobile devices are there worldwide?'
prediction = cdqa_pipeline.predict(query)

In [7]:
print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))
print('title: {}'.format(prediction[1]))
print('paragraph: {}'.format(prediction[2]))

query: How many connections to mobile devices are there worldwide?
answer: over 4 billion
title: Chapter 9 - Mobile Internet Devices and the Cloud
paragraph: 1  estimated that thereare over 4 billion connections to mobile devices worldwide—an astoundingnumber when you realize that this figure represents 60% of the global pop-ulation today. Of course, this does not mean that two out of every threepeople on Earth have a mobile phone. It is common in more than 60 coun-tries, however, for one person to have two or more devices, even while thereare no phones at all in some parts of the globe. In some countries, millionsof people are now experiencing connectivity to the world for the first timethrough wireless technologies. It is changing their economic, social, andpolitical fortunes forevermore. 


In [9]:
from cdqa.utils.converters import df2squad
# Converting dataframe to SQuAD format
json_data = df2squad(df=df, squad_version='v1.1', output_dir='.', filename='ccdatasetjson')

1it [00:00, 313.59it/s]
